# Resources:

- Countries: https://restcountries.com/v2/all

- Geocoding: https://api.api-ninjas.com/v1/geocoding

# Import

In [1]:
import requests as req

import sqlite3

import random

import time

import json

# Functions

In [2]:
def sleep_timer(min, max):
    
    """
    
        This function generates a random number between given numbers
        then sleeps.
        
        
        Example:
        
        sleep_timer(1.5, 2)
        
    """
    
    if min or max == str:
        min = int(min)
        max = int(max)
    
    sleep_sec = (max-min)*random.random() + min
    time.sleep(sleep_sec)

In [3]:
def check_key_exists(file, key):
    
    """_summary_
    This function checks if the given key exists in a data set or no,
    if not gives the indexes as a list
    """
    
    idx = []
    
    for i in range(len(file)):
        
        if key in file[i]:
            pass
        else:
            print(f"file[{i}] doesn't have any key same as: '{key}'")
            idx.append(i)
            
    return idx

# Main

### Country API

In [ ]:

# getting countries, capital, lat and lon

res_cc = req.get("https://restcountries.com/v2/all")

data_cc = res_cc.json()

In [ ]:
# writing data into json file 
with open("data/capitals.json", "w") as outfile:
    json.dump(data_cc, outfile)

In [4]:
# loading data from json file
with open("data/capitals.json", "r") as handle:
    capitals = json.loads(handle.read())


In [6]:
# Cleaning/Fixing the data

# Some countries dont have any capital or city
# so lets put the country name as substitute.

indexes = check_key_exists(capitals, "capital")

for n,i in enumerate(indexes):
    capitals[i]["capital"] = capitals[i]["name"]
    print(n+1, capitals[i]["capital"])

file[8] doesn't have any key same as: 'capital'
file[30] doesn't have any key same as: 'capital'
file[33] doesn't have any key same as: 'capital'
file[98] doesn't have any key same as: 'capital'
file[131] doesn't have any key same as: 'capital'
1 Antarctica
2 Bouvet Island
3 United States Minor Outlying Islands
4 Heard Island and McDonald Islands
5 Macao


### Getting lat and lon Then saving into database

In [7]:
# cleaning old data from DB

conn_obj = sqlite3.connect("./database/database.db")
cursor = conn_obj.cursor()

qry_delete_cc = "DELETE FROM capital_cities;"
qry_delete_ss = "DELETE FROM sqlite_sequence;"
    
cursor.execute(qry_delete_cc)
cursor.execute(qry_delete_ss)

conn_obj.close()



# Inserting new data to DB

base_url = "https://api.api-ninjas.com/v1/geocoding"

key = "Q8D8RIf37tf0zz8dY25qeA==ssFIvkwn8b7N2KoP"

for i in range(len(capitals)):
    
    # getting country and it's capital
    country = capitals[i]["name"]
    capital = capitals[i]["capital"]

    parameters = {"city":capital, "country":country}


    # getting lat and lon
    res_ll = req.get(url=base_url, params=parameters, headers={'X-Api-Key': key})
    
    if res_ll.status_code == req.codes.ok:
        for i in range(len(res_ll.json())):
            lat = res_ll.json()[i]["latitude"]
            lon = res_ll.json()[i]["longitude"]
    else:
        print("Error:", res_ll.status_code, res_ll.text)
           
           
    # saving data in DB
    
    params = (capital, country, lat, lon)
    
    conn_str = "./database/database.db"


    qry_insert = """
        INSERT INTO capital_cities (name, country, lat, lon) 
        VALUES(?, ?, ?, ?);
    """
    with sqlite3.connect(conn_str) as conn_obj:
        cursor = conn_obj.cursor()
        
        parameters = (capital, country, lat, lon)
        cursor.execute(qry_insert, parameters)
        
        print(cursor)
        
    
    # sleep
    sleep_timer(0.5, 1.5)